In [23]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
import pickle
import numpy as np  
import pandas as pd
import os

In [24]:
model = load_model('churn_model.h5')
with open('scaler.pkl','rb') as f:
    scaler = pickle.load(f)
with open('onehot_encoder_geo.pkl','rb') as f:
    label_encoder_geo = pickle.load(f)
with open('label_encoder_gender.pkl','rb') as f:
    label_encoder_gender = pickle.load(f)

In [25]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 35,
    'Tenure': 5,
    'Balance': 50000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60000
}


In [26]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

C:\Users\Vinit Solanki\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [27]:
geo_encoded

array([[1., 0., 0.]])

In [28]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,35,5,50000,2,1,1,60000


In [29]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,35,5,50000,2,1,1,60000


In [30]:
# concatination on one hot encoded data
input_df = pd.concat([input_df.drop("Geography",axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,35,5,50000,2,1,1,60000,1.0,0.0,0.0


In [32]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
input_scaled = scaler.fit_transform(input_df)
input_scaled

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [33]:
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step


array([[0.20737922]], dtype=float32)

In [34]:
prediction_prob = prediction[0][0]
if prediction_prob > 0.5:
    print(f"The customer is likely to churn with a probability of {prediction_prob:.2f}")
else:
    print(f"The customer is unlikely to churn with a probability of {prediction_prob:.2f}")

The customer is unlikely to churn with a probability of 0.21
